# Circuit Translation

In this notebook we will introduce a tool of `sqwalk` that is useful to decompose (or translate) an unitary transormation (in our case the one generated by the walker's Hamiltonian) into a series of gates that can be simulated or even run on quantum hardware. The decomposition method is based on `qiskit` thus we will need it as a dependency, in addition to our usual `SQWalker` class and some QuTiP objects.

Before jumping to the tutorial, it is useful to note here that this decomposition, for the sake of being general, it is not optimized. Indeed, while it supports any kind of quantum computer and every kind of quantum walker it usually takes up a lot of gates to implement the decomposition. To optimize the number of gate one must resort to some specific techniques in the literature that leverage the symmetries and characteristics of some particular graphs and are not general.

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import scipy

from sqwalk import SQWalker
from sqwalk import gate_decomposition
from qiskit import Aer
from qiskit.visualization import *

First we create the walker as we have seen in the previous notebooks and we run it for a certain time to have a reference result. In this case we have picked a time of 1000.

In [ ]:
#Create and run the walker
graph = nx.path_graph(8)
adj = nx.adj_matrix(graph).todense()
walker = SQWalker(np.array(adj))

time_samples = 1000
initial_node = 0
result = walker.run_walker(initial_node, time_samples)

new_state = result.final_state
nodelist = [i for i in range(adj.shape[0])]
plt.bar(nodelist, new_state.diag())
plt.show()

Our decomposition, albeit being devised as a tool to decompose walkers, can be used with any Unitary or Hamiltonian.

Note that since we will use a system of $n$ qubits, our Hamiltonian has to be $2^n$ dimensional, if the problem has the wrong dimensionality one can zero-pad it to make it work.

The time we used above in `time_samples` has to be rescaled of a factor of $100$ since the timestep of the master equation in run_walker is $10^{-2}$.

In [ ]:
#Estract the Hamiltonian from the walker
hamiltonian = walker.quantum_hamiltonian.full()

#Set the time one wants to simulate
time_rescaled = 10

#Compute the respective unitary
unitary = scipy.linalg.expm(-1j*time_rescaled*hamiltonian)

Now it is all set up to decompose our walker using the `gate_decomposition` function from `sqwalk`. To decompose it, it is suffiicent to pass our unitary to the function that, leveraging qiskit transpiling, will give us back the quantum circuit.

The `gate_decomposition` function also accepts two more arguments:
- topology: a list of connections between qubits specifying the topology of the particular hardware we want to decompose on, default topology is fully connected.
- gates: a list of allowed gates that can be used to create the decomposition, defaults to single qubit rotations and CNOT.

The resulting decomposition is a qiskit circuit object that can be exported into QASM instructions ot be executed on virtually every device.

In [ ]:
#Decompose into gates
circuit_decomp = gate_decomposition(unitary)

circuit_decomp.qasm() # port it to whatever hardware
#circuit_decomp.draw()

As an example we take a simulator backend from `qiskit` itself (it could be a real device instead of a simulator), we execute the decomposed circuit and plot the result.

In [ ]:
backend=Aer.get_backend('aer_simulator')
circuit_decomp.measure_all()
result=backend.run(circuit_decomp).result()
counts = result.get_counts(circuit_decomp)
plot_histogram(counts)

We can see that the decomposition is perfectly consistent with the quantum walker we have simulated above with SQWalk!